<a href="https://colab.research.google.com/github/shiragelb/NCC-Statistical-Reports/blob/main/alignment%20of%20tables%20and%20headers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import os
import io

# 1️⃣ Authenticate
auth.authenticate_user()
drive_service = build('drive', 'v3')

# 2️⃣ Folder ID of the specific directory
folder_id = "1xH-6eyhl0XDe0DfS33o0xZAIdTLPkyQj"  # <-- ID of "excel_and_json_2019_2021_2024"

# 3️⃣ Recursive listing of all files in this folder
def list_all_files_in_folder_recursive(parent_id, parent_path=""):
    all_files = []
    query = f"'{parent_id}' in parents and trashed=false"
    page_token = None

    while True:
        response = drive_service.files().list(
            q=query,
            spaces='drive',
            fields='nextPageToken, files(id, name, mimeType)',
            pageToken=page_token
        ).execute()

        for item in response.get('files', []):
            item_path = f"{parent_path}/{item['name']}" if parent_path else item['name']
            if item['mimeType'] == 'application/vnd.google-apps.folder':
                all_files.extend(list_all_files_in_folder_recursive(item['id'], item_path))
            else:
                all_files.append({
                    "file_name": item['name'],
                    "file_path": item_path,
                    "file_id": item['id']
                })

        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

    return all_files

# 4️⃣ Download all files to local directory
def download_files(files_list, download_dir="/content/excel_and_json_2019_2021_2024"):
    os.makedirs(download_dir, exist_ok=True)

    for file in files_list:
        local_path = os.path.join(download_dir, file['file_path'].replace("/", os.sep))
        os.makedirs(os.path.dirname(local_path), exist_ok=True)

        request = drive_service.files().get_media(fileId=file['file_id'])
        fh = io.FileIO(local_path, "wb")
        downloader = MediaIoBaseDownload(fh, request)

        done = False
        while not done:
            status, done = downloader.next_chunk()
            if status:
                print(f"⬇️ Downloading {file['file_name']} {int(status.progress() * 100)}%")

        print(f"✅ Saved {file['file_name']} to {local_path}")

# 5️⃣ Run everything
files_list = list_all_files_in_folder_recursive(folder_id)
download_files(files_list)


⬇️ Downloading 162_1_2024.csv 100%
✅ Saved 162_1_2024.csv to /content/excel_and_json_2019_2021_2024/2024/1/162_1_2024.csv
⬇️ Downloading 172_1_2024.csv 100%
✅ Saved 172_1_2024.csv to /content/excel_and_json_2019_2021_2024/2024/1/172_1_2024.csv
⬇️ Downloading 160_1_2024.csv 100%
✅ Saved 160_1_2024.csv to /content/excel_and_json_2019_2021_2024/2024/1/160_1_2024.csv
⬇️ Downloading 186_1_2024.csv 100%
✅ Saved 186_1_2024.csv to /content/excel_and_json_2019_2021_2024/2024/1/186_1_2024.csv
⬇️ Downloading 161_1_2024.csv 100%
✅ Saved 161_1_2024.csv to /content/excel_and_json_2019_2021_2024/2024/1/161_1_2024.csv
⬇️ Downloading 190_1_2024.csv 100%
✅ Saved 190_1_2024.csv to /content/excel_and_json_2019_2021_2024/2024/1/190_1_2024.csv
⬇️ Downloading 180_1_2024.csv 100%
✅ Saved 180_1_2024.csv to /content/excel_and_json_2019_2021_2024/2024/1/180_1_2024.csv
⬇️ Downloading 191_1_2024.csv 100%
✅ Saved 191_1_2024.csv to /content/excel_and_json_2019_2021_2024/2024/1/191_1_2024.csv
⬇️ Downloading 176_1_202

In [22]:
import os
import pandas as pd
import json
import re

def is_hebrew(text):
    """Return True if the text contains Hebrew letters."""
    if not isinstance(text, str):
        return False
    return bool(re.search(r'[\u0590-\u05FF]', text))

def clean_text_list(text_list):
    """Remove numbers, NaNs, and non-Hebrew/English unwanted strings."""
    cleaned = []
    for t in text_list:
        if pd.isna(t):
            continue
        t = str(t).strip()
        if not t:
            continue
        if t.lower() in ['nan', 'none']:
            continue
        # Remove numeric values
        # if re.match(r'^[-+]?[0-9]*[.,]?[0-9]+$', t):
        #     continue
        # Keep only Hebrew
        if is_hebrew(t):
            cleaned.append(t)
    return cleaned

def extract_hebrew_from_csv_per_chapter(base_dir, output_dir, rows_to_extract=3):
    """
    Extract Hebrew text from CSV files per chapter.
    Saves one JSON per chapter inside output_dir.
    """
    os.makedirs(output_dir, exist_ok=True)

    for year_folder in os.listdir(base_dir):
        year_path = os.path.join(base_dir, year_folder)
        if not os.path.isdir(year_path):
            continue

        for chapter_folder in os.listdir(year_path):
            chapter_path = os.path.join(year_path, chapter_folder)
            if not os.path.isdir(chapter_path):
                continue

            chapter_result = {}

            for file in os.listdir(chapter_path):
                if file.endswith('.csv'):
                    csv_path = os.path.join(chapter_path, file)
                    try:
                        df = pd.read_csv(csv_path, dtype=str)
                    except Exception as e:
                        print(f"Error reading {csv_path}: {e}")
                        continue

                    # Extract headers + first N rows
                    texts = []
                    if df.columns is not None:
                        texts.extend(df.columns.astype(str).tolist())
                    for i in range(min(rows_to_extract, len(df))):
                        row_text = df.iloc[i].astype(str).tolist()
                        texts.extend(row_text)

                    # Clean texts
                    cleaned_texts = clean_text_list(texts)

                    # Use filename without .csv as key
                    key = file.replace('.csv', '')
                    chapter_result[key] = cleaned_texts

            # Save JSON per chapter
            chapter_json_path = os.path.join(chapter_path, f"{chapter_folder}_{year_folder}_headers.json")
            with open(chapter_json_path, 'w', encoding='utf-8') as f:
                json.dump(chapter_result, f, ensure_ascii=False, indent=2)

            print(f"Saved chapter JSON: {chapter_json_path}")

# Example usage
base_dir = "/content/excel_and_json_2019_2021_2024/"
output_dir = "/content/hebrew_per_chapter_json/"
extract_hebrew_from_csv_per_chapter(base_dir, output_dir)


Saved chapter JSON: /content/excel_and_json_2019_2021_2024/2023/11/11_2023_headers.json
Saved chapter JSON: /content/excel_and_json_2019_2021_2024/2023/4/4_2023_headers.json
Saved chapter JSON: /content/excel_and_json_2019_2021_2024/2023/2/2_2023_headers.json
Saved chapter JSON: /content/excel_and_json_2019_2021_2024/2023/6/6_2023_headers.json
Saved chapter JSON: /content/excel_and_json_2019_2021_2024/2023/9/9_2023_headers.json
Saved chapter JSON: /content/excel_and_json_2019_2021_2024/2023/7/7_2023_headers.json
Saved chapter JSON: /content/excel_and_json_2019_2021_2024/2023/10/10_2023_headers.json
Saved chapter JSON: /content/excel_and_json_2019_2021_2024/2023/12/12_2023_headers.json
Saved chapter JSON: /content/excel_and_json_2019_2021_2024/2023/5/5_2023_headers.json
Saved chapter JSON: /content/excel_and_json_2019_2021_2024/2023/3/3_2023_headers.json
Saved chapter JSON: /content/excel_and_json_2019_2021_2024/2023/14/14_2023_headers.json
Saved chapter JSON: /content/excel_and_json_20

In [17]:
%%writefile real_embeddings.py

import numpy as np
import pickle
import os
import hashlib

try:
    from sentence_transformers import SentenceTransformer
    TRANSFORMER_AVAILABLE = True
except:
    TRANSFORMER_AVAILABLE = False
    print("Install with: !pip install sentence-transformers")

class RealEmbeddingGenerator:
    def __init__(self, model_name="sentence-transformers/LaBSE", cache_dir="cache"):
        self.cache_dir = cache_dir
        os.makedirs(cache_dir, exist_ok=True)
        self.embedding_cache = {}

        if TRANSFORMER_AVAILABLE:
            self.model = SentenceTransformer(model_name)
            self.dimension = self.model.get_sentence_embedding_dimension()
        else:
            self.model = None
            self.dimension = 768

    def get_text_hash(self, text):
        return hashlib.md5(text.encode('utf-8')).hexdigest()

    def generate_embedding(self, text, use_cache=True):
        text_hash = self.get_text_hash(text)

        if use_cache and text_hash in self.embedding_cache:
            return self.embedding_cache[text_hash]

        if self.model:
            embedding = self.model.encode(text, convert_to_numpy=True)
        else:
            # Fallback to deterministic random
            np.random.seed(int(text_hash[:8], 16) % 10000)
            embedding = np.random.randn(self.dimension)

        if use_cache:
            self.embedding_cache[text_hash] = embedding

        return embedding

    def generate_batch(self, texts, show_progress=True):
        if self.model:
            return self.model.encode(texts,
                                    batch_size=32,
                                    show_progress_bar=show_progress,
                                    convert_to_numpy=True)
        else:
            return np.array([self.generate_embedding(t) for t in texts])

    def save_cache(self):
        cache_file = os.path.join(self.cache_dir, "embedding_cache.pkl")
        with open(cache_file, 'wb') as f:
            pickle.dump(self.embedding_cache, f)


Overwriting real_embeddings.py


In [25]:
import os
import json
import numpy as np
from real_embeddings import RealEmbeddingGenerator

def cosine_similarity(a, b):
    """Compute cosine similarity between two vectors."""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-8)

def semantic_alignment_report(table_names_json_path, headers_json_path, threshold=0.7):
    """
    Compare table headers to table names using semantic similarity.
    Returns a dict with each header key mapped to all table names exceeding threshold.

    table_names_json_path: JSON with table names (keys can be wrong numbering)
    headers_json_path: JSON with table headers (correct serial numbers)
    threshold: minimum similarity to report
    """
    # Load JSONs
    with open(table_names_json_path, 'r', encoding='utf-8') as f:
        table_names = json.load(f)

    with open(headers_json_path, 'r', encoding='utf-8') as f:
        table_headers = json.load(f)

    # Initialize embedding generator
    embedder = RealEmbeddingGenerator()

    report = {}

    # Precompute embeddings for table names
    table_name_keys = list(table_names.keys())
    table_name_texts = [table_names[k] for k in table_name_keys]
    table_name_embeddings = embedder.generate_batch(table_name_texts, show_progress=False)

    def tokenize_hebrew(text):
      return [w for w in text.split() if w.strip()]

    for header_key, header_texts in table_headers.items():
        header_tokens = [token for h in header_texts for token in tokenize_hebrew(h)]
        header_embeddings = embedder.generate_batch(header_tokens, show_progress=False)

        matches = []

        for i, table_name in enumerate(table_name_texts):
            table_tokens = tokenize_hebrew(table_name)
            table_embeddings = embedder.generate_batch(table_tokens, show_progress=False)

            # Compute max similarity for each header token to any table token
            token_sims = []
            for h_emb in header_embeddings:
                sims = [cosine_similarity(h_emb, t_emb) for t_emb in table_embeddings]
                token_sims.append(max(sims))  # max similarity for this header token

            # Optional: apply weights (here uniform)
            weights = np.ones(len(token_sims))
            weighted_avg_sim = np.average(token_sims, weights=weights)

            if weighted_avg_sim >= threshold:
                matches.append({
                    "table_name_key": table_name_keys[i],
                    "table_name": table_name,
                    "similarity": float(weighted_avg_sim)
                })

        # Save all matches above threshold
        report[header_key] = matches

    return report

# Example usage:
chapter_report = semantic_alignment_report(
    "/content/excel_and_json_2019_2021_2024/2019/1/1_2019.json",
    "/content/excel_and_json_2019_2021_2024/2019/1/1_2019_headers.json",
    threshold=0.5
)
print(json.dumps(chapter_report, ensure_ascii=False, indent=2))


{
  "3_1_2019": [
    {
      "table_name_key": "2_1_2019",
      "table_name": "תרשים 1ב': יישובים* בהם ילדים מנו כ-25% ופחות או כ-50% ויותר מאוכלוסיית היישוב* / דצמבר 2018",
      "similarity": 0.5626458823680878
    },
    {
      "table_name_key": "3_1_2019",
      "table_name": "תרשים 1ג': מספר הילדים (17-0) שקיבלו במהלך השנה מעמד של תושב ארעי או תושב קבע / 2019-2018",
      "similarity": 1.0000000596046448
    },
    {
      "table_name_key": "4_1_2019",
      "table_name": "תרשים 1ד': שיעור הילדים (0- 17) הזרים שקיבלו במהלך השנה מעמד של תושב ארעי / 2018- 2019\n2019-2018",
      "similarity": 0.9200683236122131
    }
  ],
  "5_1_2019": [
    {
      "table_name_key": "1_1_2019",
      "table_name": "תרשים 1א': התפלגות הילדים בישראל, לפי דת (אחוזים) / 2018",
      "similarity": 0.5201178416609764
    },
    {
      "table_name_key": "2_1_2019",
      "table_name": "תרשים 1ב': יישובים* בהם ילדים מנו כ-25% ופחות או כ-50% ויותר מאוכלוסיית היישוב* / דצמבר 2018",
      "similarity": 0.